In [ ]:
!pip install transformers, emoji, wandb

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 880 kB 63.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8ff64d0d46b3ed2837a09c7edf2ca57056aa92a2628afdca60d300e1beeae1b3
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/685-NLP-Project/implicit-hate-detection

/content/drive/.shortcut-targets-by-id/1S8UITZ4wcfheudaDPm55HK4b4g-PPNuD/685-NLP-Project/implicit-hate-detection


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1S8UITZ4wcfheudaDPm55HK4b4g-PPNuD/685-NLP-Project/implicit-hate-detection


In [ ]:
%ls -a

 Anaconda3-5.1.0-Linux-x86_64.sh
'Copy of implicit_hate_detection_multi_class.ipynb'
 .git/
 .gitignore
 hsol_final_model.bin
'HSOL implicit_hate_detection_multi_class.ipynb'
 implicit_hate_dataloader/
 implicit_hate_detection_hate_vs_non_hate.ipynb
 implicit_hate_detection_implicit_hate_vs_non_hate.ipynb
 implicit_hate_detection.ipynb
 implicit_hate_detection_multi_class.ipynb
 implicit_hate_detection_multi.ipynb
 model.h5
 README.md
 req_instructions.md
 requirements.txt
 wandb/


In [ ]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/MyDrive/685-NLP-Project/implicit-hate-detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load implicit_hate_dataloader/dataloader.py    
%load implicit_hate_dataloader/helpers.py

In [ ]:
import os

data_path = '/content/drive/MyDrive/685-NLP-Project/implicit-hate-corpus'
dataset_filename = {
    # post (str)
    # class (str): high-level label in {`explicit_hate`,`implicit_hate`,`not_hate`}
    "stage-1": os.path.join(data_path, "implicit_hate_v1_stg1_posts.tsv"),

    # post (str)
    # class (str): fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`}
    # extra_implicit_class: (str) A secondary fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`, **None**}
    "stage-2": os.path.join(data_path, "implicit_hate_v1_stg2_posts.tsv"),

    # post (str)
    # target: (str) Free-text annotation for the group being targeted (e.g. `Black people`, `Immigrants`, etc.)
    # implied_statement: (str) Free-text annotation for the implicit or hidden underlying meaning of the post made
    # explicit (e.g. `people in minority groups are all in gangs`)
    "stage-3": os.path.join(data_path, "implicit_hate_v1_stg3_posts.tsv")
}

In [ ]:
from implicit_hate_dataloader.dataloader import Stage1Dataset, Stage2Dataset
from torch.utils.data import DataLoader

In [ ]:
import torch

implicit_hate_dataset = Stage1Dataset(dataset_filename["stage-1"], merge_hate_labels =True)

dataset_size = len(implicit_hate_dataset)
dataset_splits = [int(0.6*dataset_size), int(0.2*dataset_size), dataset_size - int(0.6*dataset_size) - int(0.2*dataset_size)]

# # Smaller dataset for testing model
# dataset_splits = [int(0.006*dataset_size), int(0.002*dataset_size), int(0.002*dataset_size)]
print("Dataset size", dataset_size)
print("Dataset splits", dataset_splits)

train_set, val_set, test_set = torch.utils.data.random_split(implicit_hate_dataset, dataset_splits)

print(train_set[0][0])
print(train_set[0][1])
print(train_set[0][2])
print(train_set[0][3])

# for i in range(30):
#   print(test_set[i][1])

train_dataloader =  DataLoader(train_set, batch_size=4, shuffle=False)
validation_dataloader = DataLoader(val_set, batch_size=4, shuffle=False)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Dataset size 21480
Dataset splits [12888, 4296, 4296]
crooked cops posted movie  take shelter  showing #endtmes visions as mental illness on my youtube feed
not_hate
tensor([  101, 15274, 10558,  6866,  3185,  2202,  7713,  4760,  1001,  2203,
        21246,  2229, 12018,  2004,  5177,  7355,  2006,  2026,  7858,  5438,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
text, label, input_ids, attention_masks, labels = next(iter(train_dataloader))
print(text, type(label), type(input_ids), type(attention_masks), type(labels))

('crooked cops posted movie  take shelter  showing #endtmes visions as mental illness on my youtube feed', 'anti-semitism is recurring problem in dutch schools via', 'i wish they would . thatbwould be one rent-collecting cuck down at election time .', "filthy jews are not white . and jews don't fight they get others too . you'll find yourself in holocaust mark 2") <class 'tuple'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
import torch
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla P100-PCIE-16GB, n_gpu: 1


In [ ]:
from torch import nn

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
import numpy as np
# function to get validation accuracy
def get_validation_performance(model, val_set, config):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    validation_dataloader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=True)

    total_correct = 0

    for batch in validation_dataloader:
      
      # Move tensors to the GPU
      b_input_ids = batch[2].to(device)
      b_input_mask = batch[3].to(device)
      b_labels = batch[4].to(device)
        
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask)

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        total_correct += torch.sum(preds == b_labels)
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

sweep_config = {
    "method" : "grid",
    "parameters" : {
        "batch_size": {
            "values" : [8]
        },
        "learning_rate" : {
            "values": [2e-5, 3e-5, 5e-5]
        },
        "epochs" : {
            "values" : [1, 2, 3, 4]
        },
        "epsilon": {
            "values" : [1e-8]
        },
    }
}

# # hyperparameters used in original paper
# sweep_config = {
#     "name": "original-ppr-hparams-sweep",
#     "method" : "grid",
#     "parameters" : {
#         "batch_size": {
#             "values" : [8, 16, 32]
#         },
#         "learning_rate" : {
#             "values": [2e-4, 3e-4, 4e-4, 2e-5]
#         },
#         "epochs" : {
#             "values" : [2, 3, 4]
#         },
#         "epsilon": {
#             "values" : [1e-8]
#         },
#     }
# }

sweep_id = wandb.sweep(sweep_config, entity="umass-iesl-is", project="cs685-project")

Create sweep with ID: 9nfybd3p
Sweep URL: https://wandb.ai/umass-iesl-is/cs685-project/sweeps/9nfybd3p


In [ ]:
from transformers import AutoModel, AutoTokenizer

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("vinai/bertweet-base")
    self.out = nn.Linear(768, 2)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False
    )
    output = pooled_output
    return self.out(output)

In [ ]:
model = SentimentClassifier(2)
model = model.to(device)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import AutoModel, AutoTokenizer
from torch import nn, optim

def train():
    with wandb.init() as run:
        config = wandb.config

        model = SentimentClassifier(2)
        model = model.to(device)

        # Tell pytorch to run this model on the GPU.
        model.cuda()
        optimizer = AdamW(model.parameters(),
                        lr = config["learning_rate"], # args.learning_rate - default is 5e-5
                        eps = config["epsilon"] # args.adam_epsilon  - default is 1e-8
                        )
        wandb.watch(model, log="all")

        for epoch_i in range(0, config["epochs"]):
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, config["epochs"]))
            print('Training...')

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode.
            model.train()

            # For each batch of training data...
            train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=False)

            for batch_idx, batch in enumerate(train_dataloader):

                # Move tensors to the GPU
                b_input_ids = batch[2].to(device)
                b_input_mask = batch[3].to(device)
                b_labels = batch[4].to(device)

                # Clear the previously calculated gradient
                model.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                outputs = model(input_ids = b_input_ids, attention_mask = b_input_mask)

                _, preds = torch.max(outputs, dim=1)
                loss = loss_fn(outputs, b_labels)

                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Update parameters and take a step using the computed gradient.
                optimizer.step()
                wandb.log({
                    "batch_loss": loss.item(),
                    "batch": batch_idx
                })
                
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set. Implement this function in the cell above.
            print(f"Total loss: {total_train_loss}")
            val_acc = get_validation_performance(model, val_set, config)
            print(f"Validation accuracy: {val_acc}")
            wandb.log({
                "loss": total_train_loss, 
                "epoch": config["epochs"], 
                "val_acc": val_acc
                })
            
        print("")
        print("Training complete!")

        test_acc = get_validation_performance(model, test_set, config)
        print(f"Test accuracy: {test_acc}")
        wandb.log({"test_acc": test_acc})

        # torch.save(model.state_dict(), "model.h5")
        # wandb.save('model.h5')

In [ ]:
count = 1 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count, entity="umass-iesl-is", project="cs685-project")

wandb: Agent Starting Run: me8j5pua with config:
wandb: 	batch_size: 8
wandb: 	epochs: 1
wandb: 	epsilon: 1e-08
wandb: 	learning_rate: 2e-05


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True`


======== Epoch 1 / 1 ========
Training...
Total loss: 989.937154814601
Validation accuracy: 0.6906424760818481

Training complete!
Test accuracy: 0.7057728171348572


batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch_loss,▄▃▆▇▆▆▃▅▃▆▆▃▂▂▂▅▃▄▃▇▆▃▅▄▄▁▄▃▅██▅▅▂▆▅▃▆▁▂
epoch,▁
loss,▁
test_acc,▁
val_acc,▁
batch,1610
batch_loss,0.5772
epoch,1
loss,989.93715
test_acc,0.70577


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, accuracy_score, precision_recall_fscore_support

final_score = precision_recall_fscore_support(preds, labels, average='binary')
class_names=['not_hate', 'hate']
wandb.log({f"conf_mat_{dataset}_{epoch}" : wandb.plot.confusion_matrix(probs=None,
                        y_true=labels, preds=preds,
                        class_names=class_names)})
cm = confusion_matrix(labels, preds, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                  display_labels=class_names)
disp.plot()

wandb.log({f"conf_mat_matplotlib_{dataset}_{epoch}": plt})
plt.show()

NameError: ignored

In [ ]:
def get_predictions(model, train_data_loader):
  model = model.eval()
  
  tweets_content = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in train_data_loader:

      print(d)
      print(d[0])

      texts = d[0]
      input_ids = d[1].to(device)
      attention_mask = d[2].to(device)
      targets = d[3].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      
      _, preds = torch.max(outputs, dim=1)
      
      probs = nn.functional.softmax(outputs, dim=1)

      tweets_content.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return tweets_content, predictions, prediction_probs, real_values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, accuracy_score, average_precision_score

def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

In [ ]:
def avg_rec(y_test, y_pred):
    rec_n, rec_u, rec_p = recall_score(y_test, y_pred, average=None)
    return (1/3) * (rec_n+ rec_u+ rec_p)

In [ ]:
def f1_np(y_test, y_pred):
    f1_n, _,f1_p = f1_score(y_test, y_pred, average=None)
    return 0.5*(f1_n+f1_p)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
model.load_state_dict(torch.load('bertweet_best_model_state.bin'))

In [ ]:
test_acc, _ = eval_model(
  model,
  validation_dataloader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()